In [1]:
!pip install mediapipe opencv-python pandas 

In [2]:
!pip install scikit-learn==1.5.1

In [4]:
import mediapipe as mp
import cv2
import numpy as np

In [5]:
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

In [6]:
VIDEO_PATH='lean.avi'
EXPORT_PATH='lean.csv'
MODEL_PATH='lean.pkl'

In [7]:
cap=cv2.VideoCapture(0)

height= cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps=cap.get(cv2.CAP_PROP_FPS)
videoWriter=cv2.VideoWriter(VIDEO_PATH,cv2.VideoWriter_fourcc('P','I','M','1'),fps,(int(width),int(height)))

while cap.isOpened():
    ret,frame=cap.read()

    try:
        cv2.imshow('videoLean',frame)
        videoWriter.write(frame)
    except Exception as e:
        break

    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
videoWriter.release()
cv2.destroyAllWindows()

In [7]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [8]:
landmarks=['class']
for val in range(1,33+1):
    landmarks +=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [7]:
with open(EXPORT_PATH,mode='w',newline='') as f:
    csv_writer=csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [8]:
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0,action)
        with open(EXPORT_PATH,mode='a',newline='') as f:
            csv_writer=csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

In [12]:
cap=cv2.VideoCapture(VIDEO_PATH)

with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,image=cap.read()

        #Recolor Feed 
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable=False

        #Make desicions
        results=pose.process(image)
        #Recolor image vack to bgr for rendering
        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                              , mp_drawing.DrawingSpec(color=(2.45,117,66), thickness=2, circle_radius=4),
                               mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                        )
        k=cv2.waitKey(1)
        if k==ord('a'): 
            export_landmark(results,'right')
        if k==ord('s'): 
            export_landmark(results,'neutral')
        if k==ord('d'): 
            export_landmark(results,'left')

        cv2.imshow('Raw Webcam Feed',image)

        if cv2.waitKey(1) & 0xFF ==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
cap.release()
cv2.destroyAllWindows()

# Train

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [10]:
df=pd.read_csv(EXPORT_PATH)

In [11]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,right,0.555137,0.178020,-0.429825,0.999938,0.564413,0.168864,-0.407780,0.999830,0.568318,...,0.102482,0.855742,0.540039,0.913449,-0.026266,0.985916,0.404056,0.921144,-0.087289,0.986604
1,left,0.558961,0.183092,-0.416674,0.999972,0.568821,0.175601,-0.392649,0.999927,0.572502,...,0.055623,0.886436,0.538719,0.910214,0.060407,0.980129,0.406523,0.923244,-0.141428,0.988544
2,left,0.535195,0.168742,-0.436347,0.999933,0.545025,0.157164,-0.415770,0.999820,0.549696,...,0.096281,0.868194,0.539584,0.911548,0.055210,0.975322,0.408350,0.923808,-0.092655,0.985141
3,neutral,0.471321,0.155718,-0.423375,0.999883,0.477337,0.139976,-0.409242,0.999577,0.482243,...,0.169757,0.827618,0.541787,0.909771,-0.016624,0.980438,0.401249,0.923728,0.000966,0.981713
4,neutral,0.469919,0.155253,-0.457150,0.999905,0.476463,0.139284,-0.442039,0.999667,0.481438,...,0.194294,0.825754,0.541361,0.909164,0.009870,0.981990,0.402319,0.923851,0.014722,0.982036


In [12]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
308,right,0.384079,0.176859,-0.288056,0.999972,0.385725,0.157554,-0.276231,0.999919,0.389805,...,0.167377,0.824152,0.543076,0.913599,-0.022381,0.968921,0.397885,0.929971,0.022423,0.976664
309,right,0.382293,0.176132,-0.290139,0.999966,0.383989,0.157486,-0.275178,0.999905,0.388287,...,0.161795,0.812479,0.541922,0.913508,-0.009354,0.962949,0.397713,0.930835,0.009321,0.973278
310,right,0.379214,0.175775,-0.349531,0.999969,0.381325,0.157203,-0.335043,0.999912,0.385736,...,0.205157,0.820017,0.541995,0.912678,0.004983,0.964327,0.399475,0.929344,0.049702,0.974538
311,right,0.376017,0.175282,-0.329445,0.999973,0.378503,0.156775,-0.316191,0.999922,0.382917,...,0.175566,0.832663,0.544801,0.913453,-0.020783,0.966644,0.401082,0.929768,0.024856,0.976763
312,right,0.376085,0.166520,-0.307183,0.999967,0.378935,0.149498,-0.295749,0.999905,0.383720,...,0.141995,0.811831,0.528941,0.913388,-0.046830,0.964154,0.405517,0.929613,-0.003891,0.973477


In [13]:
X=df.drop('class',axis=1)
y=df['class']

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1234)

In [15]:
y_test

7      neutral
40     neutral
194    neutral
187    neutral
37     neutral
        ...   
224    neutral
220    neutral
5      neutral
185    neutral
252    neutral
Name: class, Length: 94, dtype: object

# Train clasiffier

In [16]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [17]:
pipelines={
    'rf': make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(),GradientBoostingClassifier()),
}

In [18]:
fit_models={}
for algo, pipeline in pipelines.items():
    model=pipeline.fit(X_train,y_train)
    fit_models[algo]=model

In [19]:
fit_models

{'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [20]:
fit_models['rf'].predict(X_test)

array(['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'left', 'neutral', 'right', 'neutral',
       'neutral', 'neutral', 'left', 'neutral', 'left', 'left', 'right',
       'left', 'neutral', 'left', 'neutral', 'right', 'neutral', 'left',
       'left', 'neutral', 'left', 'neutral', 'left', 'left', 'neutral',
       'right', 'neutral', 'left', 'neutral', 'left', 'neutral', 'right',
       'left', 'left', 'left', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'right', 'neutral', 'neutral', 'neutral',
       'neutral', 'left', 'right', 'right', 'neutral', 'neutral', 'right',
       'neutral', 'neutral', 'right', 'left', 'left', 'neutral',
       'neutral', 'right', 'left', 'neutral', 'right', 'right', 'neutral',
       'neutral', 'left', 'right', 'neutral', 'right', 'neutral', 'left',
       'left', 'left', 'right', 'neutral', 'neutral', 'left', 'neutral',
       'right', 'neutral', 'neutral', 'neutral', 'neutral',

# Evaluate and serialize model 

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score #Accuracy metrics
import pickle

In [22]:
for algo,model in fit_models.items():
    yhat=model.predict(X_test)
    print(algo, accuracy_score(y_test.values,yhat),
         precision_score(y_test.values,yhat,average="weighted"),
         recall_score(y_test.values,yhat,average="weighted",))

rf 0.9680851063829787 0.9683060556464811 0.9680851063829787
gb 0.9680851063829787 0.9683060556464811 0.9680851063829787


In [23]:
 with open(MODEL_PATH,'wb')as f:
     pickle.dump(fit_models['rf'],f)

# Make detections

In [24]:
with open(MODEL_PATH,'rb') as f:
    model=pickle.load(f)

In [25]:
# Inicializar mediapipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Inicializar el modelo de VideoCapture
cap = cv2.VideoCapture(0)


# Inicializar el modelo de pose
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolorear el feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Hacer decisiones
        results = pose.process(image)

        # Recolorear la imagen de nuevo a BGR para renderizar
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Dibujar las landmarks de la pose
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            try:
                # Obtener las landmarks
                row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
                
                # Asegúrate de definir 'landmarks' y 'model' en tu contexto
                X = pd.DataFrame([row], columns=landmarks[1:])
                body_language_class = model.predict(X)[0]
                body_language_prob = model.predict_proba(X)[0]
               
                print(body_language_class, body_language_prob)

                if body_language_class == 'left' and body_language_prob[np.argmax(body_language_prob)] >= .7:
                    current_stage = 'left'
                elif current_stage == 'left' and body_language_class == 'right' and body_language_prob[np.argmax(body_language_prob)] <= .7:
                    current_stage = "right"
                    counter += 1
                    print(current_stage)

                # Dibujar los cuadros de las etiquetas
                cv2.rectangle(image, (0, 0), (250, 60), (245, 117, 16), -1)

                cv2.putText(image, 'CLASS', (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, body_language_class.split(' ')[0], (90, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Mostrar la probabilidad
                cv2.putText(image, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)], 2)), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                
            except Exception as e:
                print(f"Error: {e}")
                pass

        cv2.imshow('Raw Webcam Feed', image)
        # Captura la tecla 'r' para resetear el contador y el estado
        if cv2.waitKey(10) & 0xFF == ord('r'):
            counter = 0
            current_stage = ''
            print("Reset done!")

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


cap.release()
cv2.destroyAllWindows()


C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


right [0.06 0.2  0.74]
Error: name 'current_stage' is not defined
right [0.07 0.19 0.74]
Error: name 'current_stage' is not defined


C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


right [0.08 0.18 0.74]
Error: name 'current_stage' is not defined
right [0.11 0.25 0.64]
Error: name 'current_stage' is not defined


C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

left [0.35 0.31 0.34]
left [0.35 0.31 0.34]
left [0.34 0.33 0.33]


C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


left [0.34 0.33 0.33]
left [0.34 0.33 0.33]
right [0.33 0.33 0.34]


C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


neutral [0.32 0.34 0.34]
neutral [0.32 0.37 0.31]
neutral [0.32 0.37 0.31]
